In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import scipy.io as sio
import anndata as ad
import seaborn as sns
import os as os
import sys as sys
sys.path.append('/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/')
from scRNA_utils import *
import operator as op
import matplotlib.colors as mcolors
data_dir_NHDP = "/home/qiuaodon/Desktop/project_data_new/"
data_dir = "/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/CD4_CD8_final_version/"

## check the patients I can use with enough cells in subtypes


In [2]:
adata_T = sc.read(data_dir_NHDP + '1863-counts_cells_cohort1_T_cells_annotated.h5ad')

In [3]:
# get adata_CD4EX and adata_CD8EX
adata_CD4EX = adata_T[adata_T.obs['cell_type'] == 'CD4 EX', :]
adata_CD8EX = adata_T[adata_T.obs['cell_type'] == 'CD8 EX', :]

In [4]:
# check the number of cells for sample_id, find the patient_ids with more than 5 cells in both pre and on timepoints
cell_counts = adata_CD4EX.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()

# Filter patients with more than 5 cells in both 'pre' and 'on' timepoints
valid_patients_CD4EX = cell_counts[(cell_counts['pre'] > 5) & (cell_counts['on'] > 5)].index.tolist()
valid_patients_CD4EX = set(valid_patients_CD4EX)
# check the number of cells for sample_id, find the patient_ids with more than 5 cells in both pre and on timepoints in CD8EX
cell_counts = adata_CD8EX.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()

# Filter patients with more than 5 cells in both 'pre' and 'on' timepoints
valid_patients_CD8EX = cell_counts[(cell_counts['pre'] > 5) & (cell_counts['on'] > 5)].index.tolist()
valid_patients_CD8EX = set(valid_patients_CD8EX)

/tmp/ipykernel_701418/2168074212.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cell_counts = adata_CD4EX.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()
/tmp/ipykernel_701418/2168074212.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cell_counts = adata_CD8EX.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()


In [6]:
print(len(valid_patients_CD4EX), len(valid_patients_CD8EX))

16 12


In [7]:
adata_B = sc.read(data_dir_NHDP + '1863-counts_cells_cohort1_B_cells.h5ad')

In [8]:
adata_B

AnnData object with n_obs × n_vars = 8982 × 5000
    obs: 'patient_id', 'sample_id', 'timepoint', 'treatment', 'expansion', 'BC_type', 'cellType', 'n_genes', 'n_counts', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'cell_type'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'means', 'dispersions', 'dispersions_norm', 'highly_variable'
    uns: 'cell_type_colors', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'patient_id_colors', 'pca', 'rank_genes_groups', 'timepoint_colors', 'treatment_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [ ]:
# check the number of cells for sample_id, find the patient_ids with more than 5 cells in both pre and on timepoints
cell_counts = adata_B.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()

# Filter patients with more than 5 cells in both 'pre' and 'on' timepoints
valid_patients_B = cell_counts[(cell_counts['pre'] > 10) & (cell_counts['on'] > 10)].index.tolist()
valid_patients_B = set(valid_patients_B)

/tmp/ipykernel_701418/3106465870.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cell_counts = adata_M1.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()


# get the DEGs of filtered patients of CD4EX and B

In [ ]:
Patients_CD4EX_B = list(valid_patients_CD4EX & valid_patients_B)
adata_CD4EX_1 = adata_CD4EX[adata_CD4EX.obs['patient_id'].isin(Patients_CD4EX_B), :]
adata_B_1 = adata_B[adata_B.obs['patient_id'].isin(Patients_CD4EX_B), :]

In [20]:
DEG_1 = paird_ttest(adata_CD4EX_1, condition_key = 'timepoint', sample_id_col = 'sample_id', patient_id_col = 'patient_id')
DEG_1 = DEG_1[DEG_1['pval'] < 0.01]

Input adata is not pseudo-bulk RNA data. Convert to pseudo-bulk RNA data.


/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:490: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  res_df['pval'] = res_df['pval'].replace([np.inf, -np.inf], np.nan)  # Replace infinite values with NaN


In [21]:
# remove the DEGs with . in the gene names
DEG_1 = DEG_1[~DEG_1.index.str.contains('\.')]

In [ ]:
DEG_2 = paird_ttest(adata_B_1, condition_key = 'timepoint', sample_id_col = 'sample_id', patient_id_col = 'patient_id')
DEG_2 = DEG_2[DEG_2['pval'] < 0.01]

Input adata is not pseudo-bulk RNA data. Convert to pseudo-bulk RNA data.


/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:490: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  res_df['pval'] = res_df['pval'].replace([np.inf, -np.inf], np.nan)  # Replace infinite values with NaN


In [23]:
DEG_2 = DEG_2[~DEG_2.index.str.contains('\.')]

## IVA

In [24]:
adata_CD4EX_1

View of AnnData object with n_obs × n_vars = 5183 × 5000
    obs: 'patient_id', 'sample_id', 'timepoint', 'treatment', 'expansion', 'BC_type', 'cellType', 'n_genes', 'n_counts', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'cell_type'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'cell_type_colors', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [ ]:
adata_B_1

View of AnnData object with n_obs × n_vars = 7196 × 5000
    obs: 'patient_id', 'sample_id', 'timepoint', 'treatment', 'expansion', 'BC_type', 'cellType', 'n_genes', 'n_counts', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'cell_type'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'means', 'dispersions', 'dispersions_norm', 'highly_variable'
    uns: 'cell_type_colors', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'patient_id_colors', 'pca', 'rank_genes_groups', 'timepoint_colors', 'treatment_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [ ]:
adata_1_pseudo = scRNA2PseudoBulkAnnData(adata_CD4EX_1, sample_id_col='sample_id')
adata_2_pseudo = scRNA2PseudoBulkAnnData(adata_B_1, sample_id_col='sample_id')

AttributeError: 'Series' object has no attribute 'nonzero'

In [ ]:
gene_1 = DEG_1.index.tolist()
gene_2 = DEG_2.index.tolist()

# Filter out invalid gene names
gene_1 = [gene for gene in gene_1 if gene in adata_1_pseudo.var_names]
gene_2 = [gene for gene in gene_2 if gene in adata_2_pseudo.var_names]

gene_1_matrix = adata_1_pseudo[:, gene_1].X
gene_2_matrix = adata_2_pseudo[:, gene_2].X
gene_1_df = pd.DataFrame(gene_1_matrix, columns=gene_1, index=adata_1_pseudo.obs['sample_id'])
gene_2_df = pd.DataFrame(gene_2_matrix, columns=gene_2, index=adata_2_pseudo.obs['sample_id'])
gene_1_df.columns = [i + '_T' for i in gene_1_df.columns]
gene_2_df.columns = [i + '_B' for i in gene_2_df.columns]
gene_df = gene_1_df
gene_df = pd.merge(gene_df, gene_2_df, on='sample_id')
gene_df['treatment'] = gene_df.index.str.contains('On').astype(int)
# get patient id from sample id remove the _On or _Pre
gene_df['patient_id'] = gene_df.index.str.replace('_On', '').str.replace('_Pre', '')

In [30]:
# export the gene_df to csv
gene_df.to_csv(data_dir + 'gene_df_CD4EX_B.csv')

## load in IV result and perform CIT

In [50]:
DEG_pairs = pd.read_excel(data_dir + 'IV_regression_results_with_FisherZ_CD4EX_B.xlsx')

In [51]:
DEG_pairs = DEG_pairs.rename(columns = {'gene_T':'g1', 'gene_cell':'g2'})
calculate_gene_correlation(DEG_pairs, adata_1_pseudo, adata_2_pseudo)

/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:914: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.1871764908667762' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  results.loc[index, 'g1vsg2_correlation'] = corr_g1_g2


,g1,g2,p_value_iv,r_squared_iv,p_value_ci,q_value_iv,q_value_ci,g1vsg2_correlation
0,ADGRG1_T,ADPRM_B,0.318352,-2.438201,0.023314,0.323625,0.049718,-0.187176
1,ADGRG1_T,AREG_B,0.287576,-2.910224,0.013407,0.296566,0.035854,-0.138513
2,ADGRG1_T,BHLHE40_B,0.189809,-5.415265,0.000324,0.209935,0.004950,-0.150270
3,ADGRG1_T,BPNT1_B,0.192904,-1.739164,0.051021,0.212227,0.082276,-0.268889
4,ADGRG1_T,C2orf69_B,0.203998,-3.499045,0.005986,0.222092,0.022696,-0.139126
...,...,...,...,...,...,...,...,...
5273,ZFYVE16_T,TNFSF8_B,0.007683,-0.791121,0.000021,0.111097,0.000923,-0.292400
5274,ZFYVE16_T,ZBTB10_B,0.033479,-0.243247,0.032279,0.116182,0.060633,-0.258349
5275,ZFYVE16_T,ZBTB16_B,0.054917,-0.395691,0.013400,0.122188,0.035854,-0.131665
5276,ZFYVE16_T,ZNF346_B,0.082663,-0.216617,0.071133,0.131422,0.103719,-0.167093


In [52]:
# Combine the two conditions using the logical OR operator
DEG_pairs = DEG_pairs[(DEG_pairs['g1vsg2_correlation'] > 0.45) | (DEG_pairs['g1vsg2_correlation'] < -0.45)]
DEG_pairs = DEG_pairs[(DEG_pairs['p_value_ci'] > 0.1)]
# sort the DEG_pairs by g1vsg2_correlation
DEG_pairs = DEG_pairs.sort_values(by='g1vsg2_correlation', ascending=False)
DEG_pairs

,g1,g2,p_value_iv,r_squared_iv,p_value_ci,q_value_iv,q_value_ci,g1vsg2_correlation
882,CITED2_T,CENPM_B,0.000025,0.656947,0.555698,0.032803,0.570395,0.822269
980,CRYBG1_T,TLE1_B,0.000158,0.619216,0.898962,0.075335,0.901696,0.795401
4460,TAGAP_T,TLE1_B,0.000200,0.603256,0.249839,0.075335,0.275005,0.794720
3706,PMAIP1_T,TLE1_B,0.000647,0.508026,0.177966,0.087880,0.206304,0.787889
2699,KLF2_T,KLF2_B,0.000377,0.564644,0.331359,0.078530,0.352777,0.786849
...,...,...,...,...,...,...,...,...
3972,RGS1_T,INKA1_B,0.008013,0.398352,0.980914,0.111188,0.981472,-0.647392
2622,KLF10_T,CENPM_B,0.000870,0.372498,0.302153,0.090484,0.325528,-0.649447
2503,JUN_T,CD200_B,0.019697,0.106703,0.139977,0.115284,0.170113,-0.661159
2549,JUN_T,ZBTB16_B,0.019247,0.092233,0.129416,0.114883,0.160417,-0.662126


In [34]:
lrpair = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/lr_network_unique.tsv', sep='\t')

In [35]:
# remove the ppi_prediction and ppi_prediction_go
lrpair = lrpair[lrpair['database'] != 'ppi_prediction']
lrpair = lrpair[lrpair['database'] != 'ppi_prediction_go']
lrpair = lrpair[['from', 'to']]
# change from as L and to as R
lrpair.columns = ['L', 'R']
lrpair

,L,R
0,CXCL1,CXCR2
1,CXCL2,CXCR2
2,CXCL3,CXCR2
3,CXCL5,CXCR2
4,PPBP,CXCR2
...,...,...
1387,COL2A1,SDC1
1388,LAMA1,SV2A
1389,COL2A1,GP6
1390,LAMA1,DAG1


In [36]:
# Check expression in adata_1_pseudo
expressed_genes_1 = adata_1_pseudo.var_names[adata_1_pseudo.X.sum(axis=0) > 0].tolist()

# Check expression in adata_2_pseudo
expressed_genes_2 = adata_2_pseudo.var_names[adata_2_pseudo.X.sum(axis=0) > 0].tolist()

# Filter lrpair where both L and R are expressed in their respective datasets
lrpair = lrpair[
    (lrpair['L'].isin(expressed_genes_1)) & 
    (lrpair['R'].isin(expressed_genes_2))
].reset_index(drop=True)
lrpair

,L,R
0,CXCL9,CXCR3
1,CXCL10,CXCR3
2,CXCL11,CXCR3
3,CX3CL1,CX3CR1
4,CCL19,CCR7
...,...,...
152,VCAM1,ITGB1
153,VEGFC,NRP2
154,WNT5A,FZD2
155,COL2A1,ITGA1


## CIT Fisherz

In [53]:
results = CIT_test(DEG_pairs, lrpair, adata_1_pseudo, adata_2_pseudo, gene_df, method = 'fisherz', p_value_threshold=0.05)

/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/qiuaodon/anaconda3/lib

In [54]:
#calculate the correlation between g1 and g2
calculate_gene_correlation(results, adata_1_pseudo, adata_2_pseudo)
calculate_residuals_correlation(results, adata_1_pseudo, adata_2_pseudo)

/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:914: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.6033247934026146' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  results.loc[index, 'g1vsg2_correlation'] = corr_g1_g2
/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:1060: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.23278873424837604' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  results.at[index, 'g1_residualvsg2_residuals_correlation'] = corr


,g1,g2,L,R,pValue,g1vsg2_correlation,g1_residualvsg2_residuals_correlation
0,MNDA_T,MS4A6A_B,CXCL9,CXCR3,0.226601,0.603325,0.232789
1,MNDA_T,MS4A6A_B,CXCL11,CXCR3,0.052853,0.603325,0.362451
2,MNDA_T,MS4A6A_B,CCL15,CCR1,0.056798,0.603325,0.357114
3,MNDA_T,MS4A6A_B,CCL7,CCR5,0.075722,0.603325,0.334880
4,MNDA_T,MS4A6A_B,APOE,SORL1,0.125896,0.603325,0.291453
...,...,...,...,...,...,...,...
289,TSC22D3_T,IRF4_B,HBEGF,EGFR,0.144057,-0.516759,-0.278906
290,PELI1_T,EMP2_B,FN1,SDC2,0.058323,-0.519262,-0.355128
291,NFKBIA_T,IRF4_B,FASLG,FAS,0.266852,-0.522454,-0.214378
292,NFKBIA_T,FSD1L_B,FASLG,FAS,0.207405,-0.535132,-0.242332


In [55]:
results_filtered = results[(results['pValue'] > 0.1)]
results_filtered = results_filtered[(results_filtered['g1_residualvsg2_residuals_correlation'] < 0.3)]
results_filtered = results_filtered[(results_filtered['g1_residualvsg2_residuals_correlation'] > -0.3)]
results_filtered 

,g1,g2,L,R,pValue,g1vsg2_correlation,g1_residualvsg2_residuals_correlation
0,MNDA_T,MS4A6A_B,CXCL9,CXCR3,0.226601,0.603325,0.232789
4,MNDA_T,MS4A6A_B,APOE,SORL1,0.125896,0.603325,0.291453
5,EGR1_T,CAVIN1_B,C3,ITGAX,0.139203,0.577835,0.282144
7,ADGRG1_T,MS4A6A_B,CXCL9,CXCR3,0.140566,0.576510,0.281227
11,MAFB_T,MS4A6A_B,CCL7,CCR5,0.121712,0.566506,0.294526
...,...,...,...,...,...,...,...
281,BATF_T,HERC5_B,HLA-A,CD8A,0.193465,-0.502003,-0.249640
287,TSC22D3_T,IRF4_B,FASLG,FAS,0.288177,-0.516759,-0.205340
289,TSC22D3_T,IRF4_B,HBEGF,EGFR,0.144057,-0.516759,-0.278906
291,NFKBIA_T,IRF4_B,FASLG,FAS,0.266852,-0.522454,-0.214378


In [56]:
results_filtered['L_R'] = results_filtered['L'] + '_' + results_filtered['R']
results_filtered['L_R'].nunique()

37

In [57]:
results_filtered.to_csv(data_dir + 'CIT_test_results_CD4EX_B_results.csv')

In [62]:
import pandas as pd
import numpy as np

# Sample data (assuming results_filtered is already available)
# Adjusting the script to use absolute mean and determine final sign based on majority sign

grouped_results = results_filtered.groupby(["L", "R"])

# Create a new DataFrame to hold the GEM1 and GEM2 groupings
tracked_gem1 = set()
tracked_gem2 = set()

min_gene_num = 2

# Temporary list to hold data before creating DataFrame
temp_data = []

for (ligand, receptor), group in grouped_results:
    gem1 = group["g1"].drop_duplicates().tolist()
    gem2 = group["g2"].drop_duplicates().tolist()
    
    # Calculate mean correlation values using absolute mean
    g1vsg2_corr_mean = np.abs(group["g1vsg2_correlation"]).mean()
    g1_res_vs_g2_res_mean = np.abs(group["g1_residualvsg2_residuals_correlation"]).mean()
    p_value_mean = group["pValue"].mean()
    
    # Determine final sign based on majority sign of correlations
    sign_g1vsg2 = np.sign(group["g1vsg2_correlation"]).sum()
    sign_g1_res_vs_g2_res = np.sign(group["g1_residualvsg2_residuals_correlation"]).sum()
    
    final_g1vsg2_corr = g1vsg2_corr_mean * (1 if sign_g1vsg2 >= 0 else -1)
    final_g1_res_vs_g2_res = g1_res_vs_g2_res_mean * (1 if sign_g1_res_vs_g2_res >= 0 else -1)
    
    # Check the overlap between current GEM_g1 and previously tracked GEM_g1
    overlap_gem1 = len(set(gem1) & tracked_gem1)
    overlap_gem2 = len(set(gem2) & tracked_gem2)
    
    # Filter to include only those with at least three genes in GEM_g1
    if len(gem1) > min_gene_num:
        temp_data.append({
            "Ligand": ligand,
            "Receptor": receptor,
            "GEM1_elements": gem1,
            "GEM2_elements": gem2,
            "Num_genes_GEM1": len(gem1),
            "Num_genes_GEM2": len(gem2),
            "GEM1vsGEM2_corr(mean)": final_g1vsg2_corr,
            "GEM1_residualvsGEM2_residual_corr(mean)": final_g1_res_vs_g2_res,
            "average_significance": p_value_mean
        })
        # Update the tracked GEM_g1 and GEM_g2 sets
        tracked_gem1.update(gem1)
        tracked_gem2.update(gem2)

# Convert the temporary data to a DataFrame
grouped_df = pd.DataFrame(temp_data)
grouped_df

,Ligand,Receptor,GEM1_elements,GEM2_elements,Num_genes_GEM1,Num_genes_GEM2,GEM1vsGEM2_corr(mean),GEM1_residualvsGEM2_residual_corr(mean),average_significance
0,APOE,SORL1,"[MNDA_T, PAQR6_T, HECTD2_T]",[MS4A6A_B],3,1,0.505457,0.250260,0.213101
1,CCL7,CCR5,"[MAFB_T, PAQR6_T, HECTD2_T]",[MS4A6A_B],3,1,0.493184,0.283887,0.137310
2,CXCL12,CXCR3,"[IRF1_T, TNF_T, TSC22D3_T]","[ID1_B, EMP2_B]",3,2,0.482708,0.272504,0.158984
3,CXCL9,CXCR3,"[MNDA_T, ADGRG1_T, LAG3_T, F2R_T, PAQR6_T, HEC...",[MS4A6A_B],6,1,0.532547,0.239526,0.220444
4,FASLG,FAS,"[CSF1_T, TSC22D3_T, BHLHE40_T, TXNIP_T, TAGAP_...","[IRF4_B, ICAM1_B, CD69_B, CCR7_B, LMNA_B, FSD1...",7,6,-0.491233,-0.254077,0.190042
5,FN1,SDC2,"[FASLG_T, NEAT1_T, PELI1_T]",[CAVIN1_B],3,1,-0.473415,-0.272582,0.155606
6,HLA-A,CD8A,"[CCNH_T, CRYBG1_T, PMAIP1_T, BATF_T]","[ZNF346_B, HERC5_B]",4,2,-0.470696,-0.269862,0.160047
7,ICAM1,ITGAX,"[CSF1_T, BHLHE40_T, IRF1_T, TXNIP_T]",[ICAM1_B],4,1,0.489602,0.263591,0.172365
8,MPZ,MPZ,"[KLF2_T, SPON2_T, FAM13A_T]","[STAG3_B, ZBTB16_B, CNR1_B]",3,3,0.461559,0.273145,0.154715


In [63]:
# rank each GEM_g1 by order of name inside
grouped_df['GEM1_elements'] = grouped_df['GEM1_elements'].apply(lambda x: sorted(x))
grouped_df['GEM2_elements'] = grouped_df['GEM2_elements'].apply(lambda x: sorted(x))

In [64]:
# save the grouped_df to excel
grouped_df.to_excel(data_dir + 'CIT_results_CD4EXvsB_groupedLR.xlsx')